# PREPROCESSING

In [1]:
# processes a textfile according with skip-gram algorithm
# returns a list in the following format [[7,4],[42,44], ...]
from preprocessing import Preprocess
from utils import createWordPairs

# Variables: 
#     threshold: how many emojis count as a sequence
#     window_size: determine how far to the left and right of center_word the skip-gram algo forms word_pairs
threshold = 2
window_size = 8

indexes = Preprocess('./data/initializationSet.txt', threshold)
trainingCorpus = Preprocess('./data/trainingSet.txt', threshold)
validationCorpus = Preprocess('./data/validationSet.txt', threshold)

trainingPairs = createWordPairs(indexes, trainingCorpus, window_size)
validationPairs = createWordPairs(indexes, validationCorpus, window_size)

# TRAINING

In [ ]:
# best trainable after extending the data_rate_limit..
# use terminal cmd w/ MAC: "jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000"

import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader

from dataset import LoadedDataSet
from model import Word2Vec
from utils import EarlyStopping

# HyperParams
dimensionSize = 300
num_epochs = 600
lr = 0.1
batchSize = 150
patience = 25
save_name = "smalldims.w2v"
verbose = True

trainingDataset = LoadedDataSet(trainingPairs)
trainingLoader = DataLoader(trainingDataset, batchSize, shuffle=True) 
validationDataset = LoadedDataSet(validationPairs)
validationLoader = DataLoader(validationDataset, batchSize, shuffle=True) 

model = Word2Vec(indexes.vocabulary_size, dimensionSize)
optimizer = torch.optim.Adam(model.parameters(), lr)
# Variables for Scheduler:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(trainingLoader), eta_min=0, last_epoch=-1)

# initialize the early_stopping object
early_stopping = EarlyStopping(patience, verbose, save_name)
avg_train_losses = []
avg_valid_losses = []

for e in range(1, num_epochs + 1):
    train_losses = []
    valid_losses = []
    for param_group in optimizer.param_groups:
        print(param_group['lr'])    
        ###################
        # train the model #
        ###################     
    for i, (data, target) in enumerate(trainingLoader):
        model.train()
        optimizer.zero_grad()
        loss = model.forward(data, target)       
        train_losses.append(loss.item())
        loss.backward()
        optimizer.step()     
  
        ######################    
        # validate the model #
        ######################
    for i, (data, target) in enumerate(validationLoader): 
        model.eval()
        with torch.no_grad():
            loss = model.forward(data, target)       
        valid_losses.append(loss.item())

    avg_train_losses.append(np.average(train_losses))
    avg_valid_losses.append(np.average(valid_losses))
    
    scheduler.step()
    
###################################################    
#   visual Update regarding the current epoch   #
###################################################    
    epoch_len = len(str(num_epochs))
    print_msg = (f'[{e:>{epoch_len}}/{num_epochs:>{epoch_len}}] ' +
                 f'train_loss: {np.average(train_losses):.5f} ' +
                 f'valid_loss: {np.average(valid_losses):.5f}')
    print(print_msg)
    
        ##################
        # Early Stopping #
        ##################
    early_stopping(np.average(valid_losses), model)
    if early_stopping.early_stop:
        print("EARLY STOPPING!")
        break

# Visualizing the Loss and the Early Stopping Checkpoint¶

In [ ]:
import matplotlib.pyplot as plt

# visualize the loss as the network trained
fig = plt.figure(figsize=(10,8))
plt.plot(range(1,len(avg_train_losses)+1),avg_train_losses, label='Training Loss')
plt.plot(range(1,len(avg_valid_losses)+1),avg_valid_losses,label='Validation Loss')

# find position of lowest validation loss
minposs = avg_valid_losses.index(min(avg_valid_losses))+1 
plt.axvline(minposs, linestyle='--', color='r',label='Early Stopping Checkpoint')

plt.xlabel('epochs')
plt.ylabel('loss')
plt.ylim(3.5, 7) # consistent scale
plt.xlim(0, len(avg_train_losses)+1) # consistent scale
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
fig.savefig('./SOFTMAX_trained/plots/loss_plot.png', bbox_inches='tight')

# LOADING & EVALUATING TRAINED MODEL

In [4]:
# loads the weights of a saved model and calculates and prints the following metrics: SpearManRank, MSE, MAE 
# look @ the data with TensorBoardX "tensorboard --logdir runs"

import torch
import os
from evaluation import Metrics

loadedModel = torch.load(os.path.join("SOFTMAX_trained", "600epochs400dims.w2v"))
Metrics(loadedModel, indexes)

the cosine similarity between 🇬🇧 and 🇺🇸 could not be computed.
the cosine similarity between 🏅 and 🇺🇸 could not be computed.
the cosine similarity between 🇺🇸 and ❤ could not be computed.
the cosine similarity between 🇺🇸 and 💥 could not be computed.
the cosine similarity between 🎤 and 🇳🇬 could not be computed.
the cosine similarity between 🇳🇬 and 📲 could not be computed.
the cosine similarity between 👇 and 🇳🇬 could not be computed.
the cosine similarity between 🎧 and 🇳🇬 could not be computed.
the cosine similarity between 🇳🇬 and 🎶 could not be computed.
the cosine similarity between 👏 and ↪ could not be computed.

mein Spearman: 0.571855655199557
sein Spearman: 0.7590552598973165
mein MAE ist 0.38039831472298946
sein MAE ist 0.24206827309236947
mein MSE ist 0.19802350560456594
sein MSE ist 0.08023220381526104


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype <U4 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:248: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)
